In [ ]:
#This code takes a couple ionex files and combines them for later plotting. 

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import os
import re
from datetime import datetime, timedelta
from scipy.interpolate import interp1d, RegularGridInterpolator
from skimage.transform import resize
import warnings
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.animation as animation
import matplotlib.dates as mdates

warnings.filterwarnings("ignore", category=UserWarning)

# --- CONFIG ---
# IONEX Files are saved by uqrg'Day of the year'0.'year'i
#The dates used below are trying to explore Alaska event at Oasis HF station. 
IONEX_PATHS = [
    './ionosphere_central/vTEC_data/uqrg3190.12i',
    './ionosphere_central/vTEC_data/uqrg3200.12i',
    './ionosphere_central/vTEC_data/uqrg3210.12i'
]
COMBINED_IONEX_PARQUET = "combined_ionex_vtecnov2012.parquet"

def parse_ionex_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    header_end = [i for i, l in enumerate(lines) if 'END OF HEADER' in l][0]
    header = lines[:header_end]
    for line in header:
        if 'LAT1 / LAT2 / DLAT' in line:
            lat1, lat2, dlat = map(float, line.split()[:3])
        if 'LON1 / LON2 / DLON' in line:
            lon1, lon2, dlon = map(float, line.split()[:3])
    lats = np.arange(lat1, lat2 - 0.1, -abs(dlat))
    lons = np.arange(lon1, lon2 + 0.1, dlon)
    maps, epochs = [], []
    i = header_end + 1
    while i < len(lines):
        if 'START OF TEC MAP' in lines[i]:
            epoch_line = lines[i+1]
            y, mo, d, h, mi, s = map(int, epoch_line[:36].split())
            epoch = datetime(y, mo, d, h%24, mi, s) + timedelta(days=h//24)
            epochs.append(epoch)
            grid = []
            i += 2
            while 'END OF TEC MAP' not in lines[i]:
                if 'LAT/LON1/LON2/DLON/H' in lines[i]:
                    row = []
                    i += 1
                    while lines[i].strip() and 'LAT/LON1/LON2/DLON/H' not in lines[i] and 'END OF TEC MAP' not in lines[i]:
                        line_values = re.findall(r'[-+]?[0-9]*\.?[0-9]+', lines[i])
                        vals = [float(v) if v != '9999' else np.nan for v in line_values]
                        row.extend(vals)
                        i += 1
                    if len(row) == len(lons):
                        grid.append(row)
                else:
                    i += 1
            if len(grid) == len(lats):
                maps.append(grid)
        else:
            i += 1
    maps = np.array(maps)
    data = {(lat, lon): maps[:, lat_idx, lon_idx] for lat_idx, lat in enumerate(lats) for lon_idx, lon in enumerate(lons)}
    df = pd.DataFrame(data, index=epochs)
    df = 0.1 * df
    df.columns = pd.MultiIndex.from_tuples(df.columns, names=['lat', 'lon'])
    return df

def load_obs_grid(paths, save_path=None):
    dfs = []
    for fp in paths:
        if fp:
            try:
                dfs.append(parse_ionex_file(fp))
            except Exception as e:
                print(f"Failed to process {fp}: {e}")
    df = pd.concat(dfs)
    df = df[~df.index.duplicated(keep='first')].sort_index()
    if save_path:
        df.to_parquet(save_path)
    return df

obs_df = load_obs_grid(IONEX_PATHS, save_path=COMBINED_IONEX_PARQUET)